# Диагностика сердечно сосудестых заболеваний

Цель данного исследования на основе данных о пациенте предположить у него ССЗ.

## Описание даных

### Файлы

Даны три файла:
- sample_submission.csv - образец выходных данных
- test.csv - выборка для получения финальных результатов
- train.csv - выборка для тренировки модели

### Столбцы

0. id - уникальный id пациента
1. age - возраст пациента в днях
2. gender - пол
3. height - рост в см
4. weight - вес в кг
5. ap_hi - верхнее давление 
6. ap_lo - ниэнее дваление
7. cholesterol - уровень халестерина
8. gluc - 
9. smoke - курящий 
10. alco - алкоголик
11. active - активный образ жизни
12. cardio - ССЗ

## Выгрузка данных

In [2]:
import pandas as pd

In [3]:
train = pd.read_csv('./train.csv')

In [10]:
train.sample(5)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
47297,67558,19216,1,165,68.0,110,70,1,1,0,0,1,1
3238,4563,16763,1,166,64.0,110,70,1,1,0,0,1,0
31560,45060,19659,1,158,67.0,140,90,3,1,0,0,1,1
32908,47010,23247,1,156,89.0,120,80,1,1,0,0,1,1
59209,84542,19592,2,162,52.0,115,80,1,1,1,0,0,0


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           70000 non-null  int64  
 1   age          70000 non-null  int64  
 2   gender       70000 non-null  int64  
 3   height       70000 non-null  int64  
 4   weight       70000 non-null  float64
 5   ap_hi        70000 non-null  int64  
 6   ap_lo        70000 non-null  int64  
 7   cholesterol  70000 non-null  int64  
 8   gluc         70000 non-null  int64  
 9   smoke        70000 non-null  int64  
 10  alco         70000 non-null  int64  
 11  active       70000 non-null  int64  
 12  cardio       70000 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 6.9 MB
